In [156]:
import functools
from typing import Union, Callable

import clingo
from clingox.program import Program, ProgramObserver

In [157]:
static_rules = """

pos((X,Y)) :- X=1..3, Y=1..3, (X,Y) != (1,3), (X,Y) != (3,1).
adjacent((X1,Y1), (X2,Y2)) :- pos((X1,Y1)), pos((X2,Y2)), |X1 - X2| + |Y1 - Y2| = 1.

1 { observe(object(Type, Id), value(Property, Value), 0) : value(Property, Value) } 1 :- object(Type, Id).

value(at, Pos) :- pos(Pos).
action(move, Pos) :- pos(Pos).
action(wait, ()).

"""

In [158]:
effect_rules = """

observe(O, value(at, Pos2), T+1) :-
  time(T+1), time(T),
  occurs(O, action(move, Pos2), T),
  observe(O, value(at, Pos1), T),
  adjacent(Pos1, Pos2).

observe(O, value(at, Pos), T+1) :-
  time(T+1), time(T),
  occurs(O, action(wait, ()), T),
  observe(O, value(at, Pos), T).

observe(object(cell, Pos), value(collision, ()), T) :-
  observe(O1, value(at, Pos), T),
  observe(O2, value(at, Pos), T),
  O1 != O2.

observe(object(cell, Pos), value(collision, ()), T+1) :-
  O1 != O2,
  observe(O1, value(at, Pos), T),
  occurs(O2, action(move, Pos), T).

:- occurs(O, action(move, Pos2), T),                                 % Action
   time(T+1), time(T),
   observe(O, value(at, Pos1), T), adjacent(Pos1, Pos2),             % State
   observe(object(cell, Pos2), value(collision, ()), T+1).           % Precondition

"""

In [159]:
instance = """

object(robot, worker).
object(robot, charger).

init(object(robot, worker), value(at, (1,1))).
init(object(robot, charger), value(at, (2,2))).

goal(object(robot, worker), value(at, (3,3))).
goal(object(robot, charger), value(at, (2,2))).

"""

In [160]:
plan = """

time(0..k).

:- init(O, V), not observe(O, V, 0).
:- goal(O, V), not observe(O, V, k).

1 { occurs(object(ObjectType, Id), action(ActionType, Par), T) : action(ActionType, Par) } 1 :- object(ObjectType, Id), time(T), time(T+1).

"""

In [161]:
ctl = clingo.Control()
ctl.configuration.solve.models = 10
prg = Program()
obs = ProgramObserver(prg)
ctl.register_observer(obs)

In [162]:
ctl.add('base', [], static_rules)
ctl.add('base', [], effect_rules)
ctl.add('base', [], instance)
ctl.add('plan', ['k'], plan)
ctl.add('base', [], '#show observe/3. #show occurs/3.')

In [163]:
ctl.ground([("base", []), ("plan", [clingo.Number(4)])])

In [164]:
print("Facts:", len(prg.facts), "Rules:", len(prg.rules))
print(prg)

Facts: 0 Rules: 531
 :- not observe(object(robot,charger),value(at,(2,2)),4).
 :- not observe(object(robot,worker),value(at,(3,3)),4).
 :- not observe(object(robot,worker),value(at,(1,1)),0).
 :- not observe(object(robot,charger),value(at,(2,2)),0).
 :- __x13, not __x231.
 :- __x14, not __x234.
 :- __x15, not __x237.
 :- __x16, not __x240.
 :- __x17, not __x243.
 :- __x18, not __x246.
 :- __x19, not __x249.
 :- __x20, not __x252.
 :- __x42, not __x255.
 :- __x43, not __x258.
 :- observe(object(cell,(3,3)),value(collision,()),1), observe(object(robot,charger),value(at,(3,2)),0), occurs(object(robot,charger),action(move,(3,3)),0).
 :- observe(object(cell,(3,3)),value(collision,()),1), observe(object(robot,charger),value(at,(2,3)),0), occurs(object(robot,charger),action(move,(3,3)),0).
 :- observe(object(cell,(3,3)),value(collision,()),1), observe(object(robot,worker),value(at,(3,2)),0), occurs(object(robot,worker),action(move,(3,3)),0).
 :- observe(object(cell,(3,3)),value(collision,()),

In [165]:
def get_time(symbol: clingo.Symbol, default: Union[Callable[[clingo.Symbol], int], int, None] = None):
    if symbol.type is clingo.SymbolType.Function and symbol.name in ('occurs', 'observe'):
        return symbol.arguments[-1].number
    if callable(default):
        return default(symbol)
    return default

In [166]:
models = []
with ctl.solve(yield_=True) as solve_handle:
    for model in solve_handle:
        symbols = sorted(sorted(model.symbols(shown=True)), key=functools.partial(get_time, default=-1))
        print("Answer {}:{}".format(model.number,'{'), '\n'.join(map(str, symbols)), '}', sep='\n')
        models.append(symbols)
    solve_result = solve_handle.get()
    print(solve_result, end='')
    if solve_result.satisfiable:
        print(' {}{}'.format(len(models), '' if solve_result.exhausted else '+'), end='')
    print()


Answer 1:{
observe(object(robot,charger),value(at,(2,2)),0)
observe(object(robot,worker),value(at,(1,1)),0)
occurs(object(robot,charger),action(move,(1,2)),0)
occurs(object(robot,worker),action(move,(2,1)),0)
observe(object(robot,charger),value(at,(1,2)),1)
observe(object(robot,worker),value(at,(2,1)),1)
occurs(object(robot,charger),action(wait,()),1)
occurs(object(robot,worker),action(move,(2,2)),1)
observe(object(robot,charger),value(at,(1,2)),2)
observe(object(robot,worker),value(at,(2,2)),2)
occurs(object(robot,charger),action(wait,()),2)
occurs(object(robot,worker),action(move,(2,3)),2)
observe(object(robot,charger),value(at,(1,2)),3)
observe(object(robot,worker),value(at,(2,3)),3)
occurs(object(robot,charger),action(move,(2,2)),3)
occurs(object(robot,worker),action(move,(3,3)),3)
observe(object(robot,charger),value(at,(2,2)),4)
observe(object(robot,worker),value(at,(3,3)),4)
}
Answer 2:{
observe(object(robot,charger),value(at,(2,2)),0)
observe(object(robot,worker),value(at,(1,1))